In [1]:
from hmr4d import os_chdir_to_proj_root

os_chdir_to_proj_root()

In [2]:
from pathlib import Path
import torch
import numpy as np
from hmr4d.utils.wis3d_utils import make_wis3d, add_motion_as_lines

mdm_results_root = Path("inputs/mdm_results")
dataname_to_mdm_results = {
    "hml3d": np.load(mdm_results_root / "seed10_omg_hml3d/results.npy", allow_pickle=True).item(),
    "mixamo": np.load(mdm_results_root / "seed10_omg_mixamo/results.npy", allow_pickle=True).item(),
    "mtx": np.load(mdm_results_root / "seed10_omg_mtx/results.npy", allow_pickle=True).item(),
    "egoexo_release": np.load(mdm_results_root / "seed10_sample_egoexo_100v2/results.npy", allow_pickle=True).item(),
    "egoexo_gpt": np.load(mdm_results_root / "seed10_sample_gpt_egoexo_100v2_2/results.npy", allow_pickle=True).item(),
}

In [3]:
def print_info(result, name):
    print(f"==================== {name} ====================")
    print(list(result.keys()))
    num_samples = result["num_samples"]
    num_repetitions = result["num_repetitions"]
    num_motions = len(result["motion"])
    assert num_motions == num_samples * num_repetitions
    print(f"{num_motions} Motions = {num_samples} samples x {num_repetitions} repetitions")
    print(f"lenths set:", set(result["lengths"]))
    # text_lines = "\n".join([f"{i:2d}: {t}" for i, t in enumerate(sorted(set(result["text"])))])
    text_lines = "\n".join([f"{t}" for i, t in enumerate(sorted(set(result["text"])))])
    print(f"All texts:")
    # print(f"{text_lines}")


for dataname, results in dataname_to_mdm_results.items():
    print_info(results, dataname)

==================== hml3d ====================
['motion', 'text', 'lengths', 'num_samples', 'num_repetitions']
57 Motions = 19 samples x 3 repetitions
lenths set: {140}
All texts:
==================== mixamo ====================
['motion', 'text', 'lengths', 'num_samples', 'num_repetitions']
54 Motions = 18 samples x 3 repetitions
lenths set: {140}
All texts:
==================== mtx ====================
['motion', 'text', 'lengths', 'num_samples', 'num_repetitions']
54 Motions = 18 samples x 3 repetitions
lenths set: {140}
All texts:
==================== egoexo_release ====================
['motion', 'text', 'lengths', 'num_samples', 'num_repetitions']
144 Motions = 48 samples x 3 repetitions
lenths set: {140}
All texts:
==================== egoexo_gpt ====================
['motion', 'text', 'lengths', 'num_samples', 'num_repetitions']
144 Motions = 48 samples x 3 repetitions
lenths set: {140}
All texts:


In [4]:
from collections import defaultdict
from hmr4d.dataset.HumanML3D.utils import resample_motion_fps
import torch
from hmr4d.utils.wis3d_utils import make_wis3d, add_motion_as_lines
from tqdm import tqdm

dataname_to_text_info = {}

for dataname, result in tqdm(dataname_to_mdm_results.items()):
    # result = dataname_to_mdm_results["hml3d"]
    # text = result["text"]  # N*3
    # motion = result["motion"]  # N*3*196
    length = result["lengths"]  # N
    assert set(length) == {140}
    text_to_textid = {text: idx for idx, text in enumerate(sorted(set(result["text"])))}
    textid_to_text = {v: k for k, v in text_to_textid.items()}
    textid_to_motions = defaultdict(list)
    for text, motion in zip(result["text"], result["motion"]):
        textid = text_to_textid[text]
        # Clip to 140 frames and upsample from 20fps to 30fps
        motion = motion.transpose(2, 0, 1)[:140]
        larget_length = torch.tensor(140 / 20 * 30).long()
        motion = resample_motion_fps(torch.from_numpy(motion), larget_length).numpy()
        textid_to_motions[textid].append(motion)  # (F, 22, 3)

    # Add to dataname_to_text_info
    dataname_to_text_info[dataname] = {"textid_to_text": textid_to_text, "text_to_textid": text_to_textid}

    if False:
        # Add results:
        for idx in textid_to_motions:
            motions = textid_to_motions[idx]
            text = textid_to_text[idx]

            wis3d_scene_name = f"{dataname}@{idx:03d}@{text}"
            wis3d = make_wis3d(output_dir="outputs/wis3d_compare_mdm_ours", name=wis3d_scene_name)
            for i, motion in enumerate(motions):
                # Adding a offset
                motion_ = (motion + (i + 1) * np.array([1.0, 0.0, 0.0])).astype(np.float32)
                add_motion_as_lines(motion_, wis3d, name=f"mdm#{i}", const_color="red")

100%|██████████| 5/5 [00:00<00:00, 212.19it/s]


In [5]:
all_text = []
for dataname, text_info in dataname_to_text_info.items():
    all_text.extend(list(text_info["textid_to_text"].values()))
print(len(all_text))
print(len(set(all_text)))

151
151


### Ours

In [34]:
from pathlib import Path
import torch
import numpy as np
from hmr4d.utils.wis3d_utils import make_wis3d, add_motion_as_lines

ours_results_root = Path("inputs/ours_results/1008_compare3")
if False:  # convert dump_results to np_file
    dataname_to_rawdir = {
        "ours_mdm_frompred_hml4x": "outputs/dumped/mdm_frompred_hml4x",
        "ours_mdm_frompred": "outputs/dumped/mdm_frompred",
        "ours_mdm_frompred_onlyhml": "outputs/dumped/mdm_frompred_onlyhml",
    }
    for dataname, rawdir in dataname_to_rawdir.items():
        results = [torch.load(p) for p in Path(rawdir).glob("*.pth")]
        torch.save(results, ours_results_root / f"{dataname}.pt")

ours_results = {
    "ours_mdm_frompred_onlyhml": torch.load(ours_results_root / "ours_mdm_frompred_onlyhml.pt"),
    "ours_mdm_frompred_hml4x": torch.load(ours_results_root / "ours_mdm_frompred_hml4x.pt"),
    "ours_mdm_frompred": torch.load(ours_results_root / "ours_mdm_frompred.pt"),
}

In [35]:
ours_offsets = {
    "ours_mdm_frompred_onlyhml": torch.tensor([0, 0, 0]),
    "ours_mdm_frompred_hml4x": torch.tensor([-1, 0, 0]),
    "ours_mdm_frompred": torch.tensor([-2, 0, 0]),
}


# LOOK-UP INDEX
text_to_dataname_idx = {}
for dataname, text_info in dataname_to_text_info.items():
    for text, textid in text_info["text_to_textid"].items():
        text_to_dataname_idx[text] = (dataname, textid)

for method_name in ours_results:
    # method_name = "ours_mdm_frompred_hml4x"
    color = "blue"

    results = ours_results[method_name]
    for result in tqdm(results):
        # result = results[1]  # ['pred', 'text', 'length']
        pred = result["pred"]
        text = result["text"]
        dataname, idx = text_to_dataname_idx[text]

        wis3d_scene_name = f"{dataname}@{idx:03d}@{text}"
        wis3d = make_wis3d(output_dir="outputs/wis3d_compare_mdm_ours", name=wis3d_scene_name)
        pred_plot = pred + ours_offsets[method_name]
        add_motion_as_lines(pred_plot, wis3d, name=method_name, const_color=color)

100%|██████████| 151/151 [00:33<00:00,  4.53it/s]
